In [1]:
# https://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/

import pandas as pd
from datetime import datetime, timedelta, date
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import math
import matplotlib.pyplot as plt

np.random.seed(1337)

# Creating functions

In [2]:
# transform series into train and test sets for supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("var%d(t-%d)" % (j + 1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [("var%d(t)" % (j + 1)) for j in range(n_vars)]
        else:
            names += [("var%d(t+%d)" % (j + 1, i)) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    return agg


In [3]:
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
    # extract raw values
    raw_values = series.values

    # transform data to be stationary
    diff_values = raw_values
    diff_values = diff_values.reshape(len(diff_values), 1)

    # rescale values to -1, 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(diff_values)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)

    # transform into supervised learning problem X, y
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values

    # split into train and test sets
    train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
    return scaler, train, test

In [4]:
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]

    X = X.reshape(X.shape[0], 1, X.shape[1])

    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss="mean_squared_error", optimizer="adam")

    model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, verbose=2, shuffle=False)

    return model

In [5]:
# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))

    # make forecast
    forecast = model.predict(X, batch_size=n_batch)

    # convert to array
    return [x for x in forecast[0, :]]

In [6]:
# evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq, forecast_len):
    forecasts = list()
    print(f'Forecast x of {forecast_len}:', end=" ")
    for i in range(forecast_len):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # store the forecast
        forecasts.append(forecast)

        # Printing current status in hundreds
        step = i % 100
        if step == 0:
            print(i, end=" ")

    return forecasts

In [7]:
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler):
    inverted = list()
    for i in range(len(forecasts)):

        # create array from forecast
        forecast = np.array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))

        # invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]

        inverted.append(inv_scale)

    return inverted

# Fitting and predicting

In [8]:
# load dataset
logreturns = "data/final.csv"
series = pd.read_csv(logreturns, usecols=["Exchange.Date", "logreturns"], header=0, index_col=0, squeeze=True)

# configure
n_lag = 5 # same as ARMA-GARCH
n_seq = 63  #  number of periods forecast
test_share = 0.25
n_test = int(len(series) * test_share)
n_epochs = 5
n_batch = 1
n_neurons = 50
forecast_len = 1000

print("Preparing data...")
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)

print("Fitting model...")
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

print("Making forecasts...")
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq, forecast_len)

print("\nInverting forecasts...")
forecasts = inverse_transform(series, forecasts, scaler)
print("Done!")

Preparing data...
Fitting model...
Epoch 1/5
3356/3356 - 6s - loss: 0.0086
Epoch 2/5
3356/3356 - 6s - loss: 0.0078
Epoch 3/5
3356/3356 - 4s - loss: 0.0077
Epoch 4/5
3356/3356 - 4s - loss: 0.0077
Epoch 5/5
3356/3356 - 4s - loss: 0.0077
Making forecasts...
Forecast x of 1000: 0 100 200 300 400 500 600 700 800 900 
Inverting forecasts...
Done!


# Evaluating from t=1


## Creating dataframe for evaluation
In essence, creating a new DF combining training data (historic) and forecasts

In [9]:
# Getting dataframe with Close as well and the creating a training df same size as used in the model
original_df = pd.read_csv("data/final.csv", usecols=["Exchange.Date", "logreturns", "Close"])

# Setting as date
original_df['Exchange.Date'] = original_df['Exchange.Date'].apply(lambda x: date(1900, 1, 1) + timedelta(int(x)))
original_df.index = original_df['Exchange.Date']

train_df = original_df[:-n_test].copy()

# Assigning all rows in train df (before forecast) to closing value
# This is because this column cannot be empty (and we have no forecasts since it's training data)
train_df["forecast"] = train_df["Close"]

In [10]:
# Transforming logreturns back to price
last_train = train_df["Close"].values[-1]
price_forecasts = np.exp(np.cumsum(forecasts[0]) + math.log(last_train))

In [11]:
# Creating a separate dataframe only for forecasts (i.e. "outside train df")
forecast_df = pd.DataFrame(columns=["Exchange.Date", "Close", "logreturns", "forecast"])
forecast_df["Close"] = original_df["Close"].values[-n_test : -n_test + n_seq]
forecast_df["logreturns"] = original_df["logreturns"].values[-n_test : -n_test + n_seq]
forecast_df["forecast"] = price_forecasts
forecast_df.index

forecast_df["Exchange.Date"] = forecast_df.index.map(lambda x: date(2016, 8, 1) + timedelta(int(x)))
forecast_df.index = forecast_df["Exchange.Date"]

In [12]:
# Merging train and forecast dataframe
merged_df = train_df.append(forecast_df, ignore_index=True)

# Creating error, absolute error, actual price going up (True/False) and forecast going up (True/False)
merged_df["error"] = merged_df["forecast"] - merged_df["Close"]
merged_df["abs_error"] = np.abs(merged_df["forecast"] - merged_df["Close"])
merged_df["actual_up"] = merged_df["Close"].diff(1) > 0
merged_df["forecast_up"] = merged_df["forecast"].diff(1) > 0
merged_df.index = merged_df["Exchange.Date"]

# Formula for creating confusion value, used below
def confusion(actual, forecast):
    if actual and forecast:
        return "TP"

    if actual and not forecast:
        return "FN"

    if not actual and forecast:
        return "FP"

    if not actual and not forecast:
        return "TN"

    # Just common programming sense to return something, could have written "blabla"
    return False


# The lambda stuff applies the above function on every row of data
merged_df["confusion"] = merged_df.apply(lambda x: confusion(x["actual_up"], x["forecast_up"]), axis=1)

# Printing the tail of the data
merged_df.tail()

,Exchange.Date,Close,logreturns,forecast,error,abs_error,actual_up,forecast_up,confusion
Exchange.Date,,,,,,,,,
2016-09-28,2016-09-28,671.89,0.005567,655.097839,-16.792161,16.792161,True,True,TP
2016-09-29,2016-09-29,671.08,-0.001206,656.174255,-14.905745,14.905745,False,True,FP
2016-09-30,2016-09-30,668.82,-0.003373,657.349854,-11.470146,11.470146,False,True,FP
2016-10-01,2016-10-01,668.02,-0.001197,658.540466,-9.479534,9.479534,False,True,FP
2016-10-02,2016-10-02,663.33,-0.007046,659.892822,-3.437178,3.437178,False,True,FP


## Evaluating

In [13]:
# New dataframe that only contains the number of periods to evaluate (1,3,5,21,63)
def new_df(n_periods):
    df = merged_df[len(train_df) : len(train_df) + n_periods]
    return df

In [14]:
# Creating RMSE AND MAE
def evaluate(n_periods):
    df = new_df(n_periods)
    mape = ((df["abs_error"] / df["Close"]).sum() / n_periods) * 100
    rmse = math.sqrt(pow(df["error"].sum(), 2) / n_periods)
    print(f"{n_periods}, RMSE: {round(rmse, 3)}, MAPE: {round(mape, 3)}%")


evaluate(1)  # 1 day
evaluate(3)  # half a week
evaluate(5)  # week
evaluate(21)  # month
evaluate(63)  # quarter

1, RMSE: 0.648, MAPE: 0.104%
3, RMSE: 6.319, MAPE: 0.66%
5, RMSE: 8.584, MAPE: 0.661%
21, RMSE: 5.722, MAPE: 0.478%
63, RMSE: 44.001, MAPE: 1.174%


In [29]:
# Creating confusion matrix
def confusion_matrix(df):
    conf = pd.DataFrame(columns=["P", "N"], index=["P", "N"])
    conf.loc["P", "P"] = len(df[df["confusion"] == "TP"])
    conf.loc["P", "N"] = len(df[df["confusion"] == "FN"])
    conf.loc["N", "P"] = len(df[df["confusion"] == "FP"])
    conf.loc["N", "N"] = len(df[df["confusion"] == "TN"])
    return conf


confusion = confusion_matrix(new_df(3))
precision = confusion.iloc[0, 0] / (confusion.iloc[0, 0] + confusion.iloc[1, 0])
recall = confusion.iloc[0, 0] / (confusion.iloc[0, 0] + confusion.iloc[0, 1])
f_score = 2 * precision * recall / (precision + recall)

print(confusion)
print(f"precision: {int(precision*100)}%, recall: {int(recall*100)}%, f-score: {round(f_score, 3)}")

   P  N
P  1  0
N  2  0
precision: 33%, recall: 100%, f-score: 0.5


# Plotting

In [ ]:
plot_df = merged_df[-n_seq - (n_seq * 2) :]
plt.figure(figsize=(10, 5))
plt.plot(plot_df["forecast"], label="forecast")
plt.plot(plot_df["Close"], label="actual")
plt.legend()

# Cross-validating 2.0
The above one is messy, this is (hopefully) better. And more like arch_evaluate

## Generating CSV
_Not recommended to run this chunk, use the one below (this one takes ~ 5 minutes to finish, since it's creating the csv)_

In [24]:
# Creating df that is as alike arch_evaluate as possible
original_df = pd.read_csv("data/final.csv", usecols=["Close"])
cross_df = pd.DataFrame(columns=['time', 'forecast', 'Close'])

forecast_len = 1000
for i in range(forecast_len):
    train_size_cv = int(len(original_df) * 0.75) + i
    train_cv = original_df[:train_size_cv]
    last_train = train_cv.values[-1]
    test_cv = original_df[train_size_cv : len(original_df)]

    price_forecasts = np.exp(np.cumsum(forecasts[i]) + math.log(last_train))

    for j, forecast in enumerate(price_forecasts):
        cross_df = cross_df.append({
            'time': i+1,
            'forecast': forecast,
            'Close': test_cv['Close'].values[j]
        }, ignore_index=True)

    step = i % 100
    if step == 0:
        print(i, end=" ")

print('done!')
cross_df.to_csv('data/lstm_cross_val.csv', index=False)

0 100 200 300 400 500 600 700 800 900 done!


## Reading CSV and assigning columns
_Run this chunk instead, it reads the csv from the chunk above_

In [26]:
# Creating dataframe columns (error, absolute error, actual_up, forecast_up och confusion (TP, FP, TN, FN))
lstm_cross_df = pd.read_csv('data/lstm_cross_val.csv')

# adding first row of data based on last row of test data
new_data = []
new_data.insert(0, {'time':0, 'Close': 621.38, 'forecast': 621.38})
lstm_cross_df = pd.concat([pd.DataFrame(new_data), lstm_cross_df], ignore_index=True)

# creating error and up columns 
lstm_cross_df['error'] = lstm_cross_df['forecast'] - lstm_cross_df['Close']
lstm_cross_df['abs_error'] = np.abs(lstm_cross_df['forecast'] - lstm_cross_df['Close'])
lstm_cross_df['actual_up'] = lstm_cross_df['Close'].diff(1) > 0
lstm_cross_df['forecast_up'] = lstm_cross_df['forecast'].diff(1) > 0

def confusion(actual, forecast):
    if (actual and forecast):
        return 'TP'
    
    if (actual and not forecast):
        return 'FN'
    
    if (not actual and forecast):
        return 'FP'
    
    if (not actual and not forecast):
        return 'TN'
    
    return False

lstm_cross_df['confusion'] = lstm_cross_df.apply(lambda x: confusion(x['actual_up'], x['forecast_up']), axis=1)

lstm_cross_df.head(5)

,time,Close,forecast,error,abs_error,actual_up,forecast_up,confusion
0,0.0,621.38,621.380000,0.000000,0.000000,False,False,TN
1,1.0,622.77,622.121887,-0.648113,0.648113,True,True,TP
2,1.0,618.70,623.105164,4.405164,4.405164,False,True,FP
3,1.0,617.12,624.307251,7.187251,7.187251,False,True,FP
4,1.0,621.28,625.417114,4.137114,4.137114,True,True,TP


## Creating cross evaluated columns

In [27]:
def cross_evaluate(df, n_periods):
    df = df[-63:-63+n_periods] if n_periods < 63 else df.tail(63)
    mape = ((df["abs_error"] / df["Close"]).sum() / n_periods) * 100
    rmse = math.sqrt(pow(df["error"].sum(), 2) / n_periods)

    tp = len(df[df['confusion'] == 'TP'])
    fp = len(df[df['confusion'] == 'FP'])
    fn = len(df[df['confusion'] == 'FN'])

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0 # if else för att undvika division by zero errror
    recall = tp / (tp + fn) if (tp + fn > 0) else 0
    fscore = (2*precision*recall)/(precision+recall) if (precision + recall > 0) else 0

    return mape, rmse, precision, recall, fscore

cross_df = pd.DataFrame(columns=[
    "mape_1", 
    "mape_3",
    "mape_5",
    "mape_21",
    "mape_63",
    "rmse_1",
    "rmse_3",
    "rmse_5",
    "rmse_21",
    "rmse_63",
    'precision_1',
    'precision_3',
    'precision_5',
    'precision_21',
    'precision_63',
    'recall_1',
    'recall_3',
    'recall_5',
    'recall_21',
    'recall_63',
    'fscore_1',
    'fscore_3',
    'fscore_5',
    'fscore_21',
    'fscore_63',
])

forecast_len = 1000
for i in range(forecast_len):
    cross_merged_df = lstm_cross_df[i+1 : i+63+1] # to avoid first row 1 is added (which contains last training)
    one = cross_evaluate(cross_merged_df, 1)
    three = cross_evaluate(cross_merged_df, 3)
    five = cross_evaluate(cross_merged_df, 5)
    twentyone = cross_evaluate(cross_merged_df, 21)
    sixtythree = cross_evaluate(cross_merged_df, 63)

    cross_df = cross_df.append({
        'mape_1': one[0],
        'mape_3': three[0],
        'mape_5': five[0],
        'mape_21': twentyone[0],
        'mape_63': sixtythree[0],
        'rmse_1': one[1],
        'rmse_3': three[1],
        'rmse_5': five[1],
        'rmse_21': twentyone[1],
        'rmse_63': sixtythree[1],
        'precision_1': one[2],
        'precision_3': three[2],
        'precision_5': five[2],
        'precision_21': twentyone[2],
        'precision_63': sixtythree[2],
        'recall_1': one[3],
        'recall_3': three[3],
        'recall_5': five[3],
        'recall_21': twentyone[3],
        'recall_63': sixtythree[3],
        'fscore_1': one[4],
        'fscore_3': three[4],
        'fscore_5': five[4],
        'fscore_21': twentyone[4],
        'fscore_63': sixtythree[4],
    }, ignore_index=True)

cross_df.head(1) # notera hur raden här är identisk med resultatet när vi inte körde korsvalidering

,mape_1,mape_3,mape_5,mape_21,mape_63,rmse_1,rmse_3,rmse_5,rmse_21,rmse_63,...,recall_1,recall_3,recall_5,recall_21,recall_63,fscore_1,fscore_3,fscore_5,fscore_21,fscore_63
0,0.104069,0.660239,0.66148,0.477608,1.17385,0.648113,6.318696,8.58361,5.722093,44.000889,...,1.0,1.0,1.0,1.0,0.972973,1.0,0.5,0.75,0.764706,0.727273


## Description of RMSE and MAPE

In [30]:
cross_df.iloc[:, :10].describe()

,mape_1,mape_3,mape_5,mape_21,mape_63,rmse_1,rmse_3,rmse_5,rmse_21,rmse_63
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,2.658915,2.664283,2.669997,2.694996,2.713831,17.597263,30.287632,38.962604,79.783066,139.389124
std,1.975212,1.917600,1.871873,1.570899,0.920836,13.300598,22.604696,28.688250,50.138198,51.022857
min,0.000251,0.025409,0.111470,0.405763,0.929889,0.001597,0.000309,0.053601,0.032072,37.661023
25%,0.890477,0.920497,0.926656,1.331999,1.945007,5.711175,10.076860,12.753381,37.575051,100.083634
50%,2.199088,2.226364,2.215159,2.399267,2.764713,14.252118,24.856386,32.151016,72.517032,142.228716
75%,4.311375,4.287757,4.324418,3.921087,3.628557,28.566812,49.420654,64.186421,119.145236,189.809872
max,7.057885,6.828700,6.745545,6.182137,4.180595,47.637195,79.638420,101.418766,189.323809,220.316387


## Description of precision, recall and fscore

In [32]:
cross_df.iloc[:, 10:].describe()

,precision_1,precision_3,precision_5,precision_21,precision_63,recall_1,recall_3,recall_5,recall_21,recall_63,fscore_1,fscore_3,fscore_5,fscore_21,fscore_63
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.405000,0.464167,0.490250,0.559407,0.560880,0.405000,0.668833,0.739083,0.736007,0.729661,0.405000,0.513833,0.544937,0.592908,0.626730
std,0.491138,0.370424,0.321948,0.199862,0.048775,0.491138,0.447960,0.396280,0.294759,0.155639,0.491138,0.364848,0.311632,0.190749,0.073600
min,0.000000,0.000000,0.000000,0.000000,0.476190,0.000000,0.000000,0.000000,0.000000,0.441176,0.000000,0.000000,0.000000,0.000000,0.468750
25%,0.000000,0.000000,0.200000,0.444444,0.520000,0.000000,0.000000,0.500000,0.538462,0.617647,0.000000,0.000000,0.333333,0.518519,0.571429
50%,0.000000,0.333333,0.600000,0.575188,0.563636,0.000000,1.000000,1.000000,0.857143,0.714286,0.000000,0.500000,0.666667,0.640000,0.607595
75%,1.000000,0.666667,0.750000,0.684211,0.600000,1.000000,1.000000,1.000000,1.000000,0.861111,1.000000,0.800000,0.750000,0.733333,0.695652
max,1.000000,1.000000,1.000000,1.000000,0.659574,1.000000,1.000000,1.000000,1.000000,0.973684,1.000000,1.000000,1.000000,0.857143,0.756098


## Confidence intervals for RMSE, MAPE, precision, recall and fscore

In [34]:
n = cross_df.count()[0]
mean = cross_df.mean()
upper = cross_df.mean() + 1.96 * cross_df.std() / math.sqrt(n)
lower = cross_df.mean() - 1.96 * cross_df.std() / math.sqrt(n)

ci_df = pd.DataFrame(columns=['measure', 'mean', 'lower', 'upper'])

for i in range(25):
    ci_df = ci_df.append({
        'measure': cross_df.columns[i],
        'mean': mean[i],
        'lower': lower[i],
        'upper': upper[i]
    }, ignore_index=True)

ci_df

,measure,mean,lower,upper
0,mape_1,2.658915,2.536490,2.781340
1,mape_3,2.664283,2.545429,2.783137
2,mape_5,2.669997,2.553977,2.786017
3,mape_21,2.694996,2.597631,2.792362
4,mape_63,2.713831,2.656757,2.770905
5,rmse_1,17.597263,16.772883,18.421643
6,rmse_3,30.287632,28.886579,31.688686
7,rmse_5,38.962604,37.184488,40.740721
8,rmse_21,79.783066,76.675468,82.890664
9,rmse_63,139.389124,136.226694,142.551553
